In [5]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

client = Socrata("data.cdc.gov", None)

# Initialize empty list to store all results
all_results = []

# Parameters for pagination
offset = 0
limit = 10000

while True:
    # Fetch a chunk of data
    results_chunk = client.get("jr58-6ysp", content_type="json", select="variant, share, week_ending, usa_or_hhsregion", limit=limit, offset=offset)
    
    # If chunk is empty or smaller than limit, stop fetching
    if not results_chunk or len(results_chunk) < limit:
        all_results.extend(results_chunk)
        break
    
    # Otherwise, add chunk to all results and increment the offset
    all_results.extend(results_chunk)
    print('running...')
    offset += limit

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(all_results)

results_df.tail(5)

running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...
running...

,variant,share,week_ending,usa_or_hhsregion
1078797,BN.1,0.023594103753566742,2022-11-19T00:00:00.000,10
1078798,BQ.1,0.20200251042842865,2022-11-19T00:00:00.000,10
1078799,BQ.1.1,0.17415671050548553,2022-11-19T00:00:00.000,10
1078800,Other,0.0,2022-11-19T00:00:00.000,10
1078801,XBB,0.028597155585885048,2022-11-19T00:00:00.000,10


In [7]:
# Create a column for the prefix only
results_df['variant_prefix'] = results_df['variant'].str.split('.').str[0]
results_df.tail(5)

# Convert the appropriate columns to python datetime
results_df['week_ending'] = pd.to_datetime(results_df['week_ending'], format='%Y-%m-%dT%H:%M:%S.%f')
results_df.tail(5)

,variant,share,week_ending,usa_or_hhsregion,variant_prefix
1078797,BN.1,0.023594103753566742,2022-11-19,10,BN
1078798,BQ.1,0.20200251042842865,2022-11-19,10,BQ
1078799,BQ.1.1,0.17415671050548553,2022-11-19,10,BQ
1078800,Other,0.0,2022-11-19,10,Other
1078801,XBB,0.028597155585885048,2022-11-19,10,XBB


In [8]:
# rename chare column then convert to floats 
results_df.rename(columns={'share' : 'Share (%)'}, inplace=True)
results_df['Share (%)'] = results_df['Share (%)'].astype(float) * 100
results_df.head(5)

,variant,Share (%),week_ending,usa_or_hhsregion,variant_prefix
0,CH.1.1,0.261089,2023-02-25,1,CH
1,BN.1,0.046195,2023-02-25,1,BN
2,XBB.1.5.1,1.951178,2023-02-25,1,XBB
3,BA.5.2.6,0.006352,2023-02-25,1,BA
4,BA.2.75,0.003724,2023-02-25,1,BA


,variant,share,week_ending,usa_or_hhsregion
1078797,BN.1,0.023594103753566742,2022-11-19,10
1078798,BQ.1,0.20200251042842865,2022-11-19,10
1078799,BQ.1.1,0.17415671050548553,2022-11-19,10
1078800,Other,0.0,2022-11-19,10
1078801,XBB,0.028597155585885048,2022-11-19,10


In [24]:
# results_df.tail()

In [10]:
# results_df['variant'].nunique()

54